In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

In [ ]:
# Load the pre-trained model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(36, activation='softmax'))

# Load the saved weights
model.load_weights('../weights/license_plate_model_weights.h5')

In [ ]:
# Function to segment characters from the license plate image
def segment_characters(image):
    # Resize the license plate image
    img_lp = cv2.resize(image, (333, 75))
    
    # Convert to grayscale
    img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY)
    
    # Display the grayscale image
    plt.figure()
    plt.imshow(img_gray_lp, cmap='gray')
    plt.title('Grayscale License Plate')
    plt.axis('off')
    plt.show()
    
    # Binarize the image
    _, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Display the binarized image
    plt.figure()
    plt.imshow(img_binary_lp, cmap='gray')
    plt.title('Binarized License Plate')
    plt.axis('off')
    plt.show()
    
    # Apply morphological transformations
    img_binary_lp = cv2.erode(img_binary_lp, (3, 3))
    img_binary_lp = cv2.dilate(img_binary_lp, (3, 3))
    
    # Display the morphed image
    plt.figure()
    plt.imshow(img_binary_lp, cmap='gray')
    plt.title('Morphological Transformations (Erosion + Dilation)')
    plt.axis('off')
    plt.show()
    
    LP_WIDTH = img_binary_lp.shape[0]
    LP_HEIGHT = img_binary_lp.shape[1]

    # Make borders white to avoid false contour detection at edges
    img_binary_lp[0:3, :] = 255
    img_binary_lp[:, 0:3] = 255
    img_binary_lp[72:75, :] = 255
    img_binary_lp[:, 330:333] = 255
    
    # Estimate dimensions for contours
    dimensions = [LP_WIDTH/6, LP_WIDTH/2, LP_HEIGHT/10, 2*LP_HEIGHT/3]
    
    # Find contours
    char_list = find_contours(dimensions, img_binary_lp)
    
    return char_list

In [ ]:
# Function to match contours to character templates
def find_contours(dimensions, img):
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    lower_width, upper_width, lower_height, upper_height = dimensions
    
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
    x_cntr_list = []
    img_res = []
    for cntr in cntrs:
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
        
        if lower_width < intWidth < upper_width and lower_height < intHeight < upper_height:
            x_cntr_list.append(intX)
            char_copy = np.zeros((44, 24))
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))
            char = cv2.subtract(255, char)
            char_copy[2:42, 2:22] = char
            img_res.append(char_copy)
    
    # Sort characters by x-coordinate
    indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_copy = [img_res[idx] for idx in indices]
    
    # Display detected contours
    plt.figure()
    plt.imshow(img, cmap='gray')
    plt.title('Character Contours')
    plt.axis('off')
    plt.show()
    
    return np.array(img_res_copy)

In [ ]:
# Helper function to format the characters for model input
def fix_dimension(img):
    new_img = np.zeros((28, 28, 3))
    for i in range(3):
        new_img[:, :, i] = img
    return new_img

In [ ]:
# Function to show results after character recognition
def show_results(char_list):
    dic = {i: c for i, c in enumerate('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')}
    output = []
    
    for ch in char_list:
        img = cv2.resize(ch, (28, 28), interpolation=cv2.INTER_AREA)
        img = fix_dimension(img)
        img = img.reshape(1, 28, 28, 3)
        y_ = model.predict(img)[0]
        character = dic[np.argmax(y_)]
        output.append(character)
    
    plate_number = ''.join(output)
    return plate_number

In [ ]:
# Load the license plate image directly
license_plate_img = cv2.imread('../images/input/plate.png')

# Display the original license plate image
plt.imshow(cv2.cvtColor(license_plate_img, cv2.COLOR_BGR2RGB))
plt.title('Input License Plate')
plt.axis('off')
plt.show()

# Segment characters from the license plate
char_list = segment_characters(license_plate_img)

# Display segmented characters
plt.figure(figsize=(10, 6))
for i, ch in enumerate(char_list):
    plt.subplot(1, len(char_list), i + 1)
    plt.imshow(ch, cmap='gray')
    plt.title(f'{i + 1}')
    plt.axis('off')
plt.show()

# Predict characters
plate_number = show_results(char_list)
print(f"Predicted License Plate Number: {plate_number}")